In [21]:
import json
import requests
from bs4 import BeautifulSoup, Comment
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

def extract_html_from_url(url):
    try:
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        
        driver = webdriver.Chrome(
            service=Service(ChromeDriverManager().install()), 
            options=chrome_options
        )
        
        driver.get(url)
        html_content = driver.page_source
        driver.quit()
        return html_content
            
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def clean_html_content(html):
    """
    Nettoyer le contenu HTML en supprimant les éléments masqués
    et les sections non pertinentes pour le français.
    """
    soup = BeautifulSoup(html, 'html.parser')

    # Supprimer les balises inutiles
    for element in soup(["script", "style", "head", "header", "footer", "nav", "aside", "noscript"]):
        element.decompose()

    # Supprimer les commentaires HTML
    for comment in soup.find_all(text=lambda text: isinstance(text, Comment)):
        comment.extract()

    # Supprimer les éléments masqués
    for hidden_element in soup.find_all(style=lambda value: value and ('display:none' in value or 'visibility:hidden' in value)):
        hidden_element.decompose()
    for hidden_class in soup.find_all(attrs={"class": lambda value: value and 'hidden' in value.lower()}):
        hidden_class.decompose()

    # Supprimer les sections avec des mots-clés non pertinents
    mots_cles_a_ignorer = [
        'cookies', 'cookie', 'politique de confidentialité', 'conditions générales', 
        'mentions légales', 'cgu', 'cgv', 'données personnelles', 'protection des données'
    ]
    for element in soup.find_all(text=True):
        if any(mot in element.lower() for mot in mots_cles_a_ignorer):
            parent = element.find_parent()
            if parent:
                parent.decompose()

    return soup


def extract_sections_by_titles(soup):
    """
    Extraire les sections structurées par titre en français,
    tout en excluant le titre du contenu.
    """
    header_tags = ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']
    sections = []

    all_elements = iter(soup.find_all())
    last_section = None

    for element in all_elements:
        if element.name in header_tags:
            # Titre trouvé
            title = element.get_text(strip=True)

            # Ignorer les titres vides
            if not title:
                continue

            # Préparer une nouvelle section
            last_section = {'title': title, 'content': ''}
            sections.append(last_section)
        elif last_section:
            # Ajouter du contenu à la section actuelle
            if hasattr(element, 'get_text') and element not in last_section['title']:
                text = element.get_text(separator=' ', strip=True)
                if text:
                    last_section['content'] += ' ' + text

    # Nettoyage final : Supprimer les sections avec des titres vides ou dupliqués
    cleaned_sections = []
    for section in sections:
        title = section['title'].strip()
        content = section['content'].strip()

        if not title and content:
            # Fusionner avec la section précédente si possible
            if cleaned_sections:
                cleaned_sections[-1]['content'] += ' ' + content
        elif title and content:
            cleaned_sections.append({'title': title, 'content': content})

    return cleaned_sections



def main():
    # Exemple d'URL
    url = "https://www.palais-decouverte.fr/fr/explorer-nos-contenus/dessine-moi-les-sciences/le-clitoris"
    html = extract_html_from_url(url)
    
    if html:
        # Nettoyer le contenu HTML
        soup = clean_html_content(html)
        
        # Extraire les sections par titres
        sections = extract_sections_by_titles(soup)
        
        # Afficher les sections
        for section in sections:
            print("Title:", section['title'])
            print("Content:", section['content'])
            print("-" * 50)

if __name__ == "__main__":
    main()


IndentationError: expected an indented block (3723431793.py, line 97)